In [36]:
folders = []
for i in range(0,4): folders.append(f"revacc_{i}")
folders

['revacc_0', 'revacc_1', 'revacc_2', 'revacc_3']

In [37]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import os
import glob
import time 
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from PIL import Image
from sklearn.model_selection import train_test_split

from src.data import LungDataset, blend, Pad, Crop, Resize, Rotate
from src.models import UNet, PretrainedUNet
from src.metrics import jaccard, dice

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


device(type='cuda', index=0)

In [38]:
unet = PretrainedUNet(1, 2, True, "bilinear")

In [39]:

#now load the original training data we have GT for 
data_folder = Path("/media/rolyn/Data/studi/MA/dataset") # local
origins_folder = data_folder / "images"
masks_folder = data_folder / "masks"
models_folder = Path("models")
images_folder = Path("images")

origins_list = [f.stem for f in origins_folder.glob("*.png")]
origins_list = [o for o in origins_list if not o.startswith("._")] # remove weird ._ files
masks_list = [f.stem for f in masks_folder.glob("*.png")]
masks_list = [o for o in masks_list if not o.startswith("._")] # remove weird ._ files

print(len(origins_list))
print(len(masks_list))

origin_mask_list = [(mask_name.replace("_mask", ""), mask_name) for mask_name in masks_list]
print(len(origin_mask_list))
original_splits = {}
original_splits["train"], original_splits["test"] = train_test_split(origin_mask_list, test_size=0.2, random_state=42)

val_test_transforms = torchvision.transforms.Compose([
    Resize((512, 512)),
])

train_transforms = torchvision.transforms.Compose([
    Pad(300),
    Rotate(-25, 25),
    Crop(300),
    val_test_transforms,
])

datasets = {x: LungDataset(
    original_splits[x], 
    origins_folder, 
    masks_folder, 
    train_transforms if x == "train" else val_test_transforms
) for x in ["train", "test"]}

dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=1) for x in ["train", "test"]}

800
704
704


In [40]:
def evaluate(folder, resdf:pd.DataFrame(), dataloaders):
    model_name = f"{folder}_unet-6v.pt"

    models_folder = Path("models")


    #model_name = "os_unet_v1.pt"
    print(model_name)
    unet.load_state_dict(torch.load(models_folder / model_name, map_location=torch.device("cpu")))
    unet.to(device)
    #unet.eval()

    test_loss = []
    test_jaccard = []
    test_dice = []

    for origins, masks in dataloaders["test"]:
        num = origins.size(0)
        #print(num)

        origins = origins.to(device)
        masks = masks.to(device)

        with torch.no_grad():
            outs = unet(origins)
            softmax = torch.nn.functional.log_softmax(outs, dim=1)
            test_loss.append(torch.nn.functional.nll_loss(softmax, masks).item() * num)

            outs = torch.argmax(softmax, dim=1)
            outs = outs.float()
            masks = masks.float()
            test_jaccard.append(jaccard(masks, outs).item() * num)
            test_dice.append(dice(masks, outs).item() * num)
            #test_jaccard += jaccard(masks, outs).item() * num
            #test_dice += dice(masks, outs).item() * num
        print(".", end="")

    #test_loss = test_loss / len(datasets["test"])
    #test_jaccard = test_jaccard / len(datasets["test"])
    #test_dice = test_dice / len(datasets["test"])

    m_test_loss = np.mean(test_loss)
    m_test_jaccard = np.mean(test_jaccard)
    m_test_dice = np.mean(test_dice)

    print()
    print("Scores on {} test images: ".format(len(datasets["test"])))
    print(f"avg test loss: {m_test_loss}")
    print(f"avg test jaccard: {m_test_jaccard}")
    print(f"avg test dice: {m_test_dice}")

    resdf.at[folder, "loss"] = m_test_loss
    resdf.at[folder, "loss_sd"] = np.std(test_loss)
    resdf.at[folder, "jaccard"] = m_test_jaccard
    resdf.at[folder, "jaccard_sd"] = np.std(test_jaccard)
    resdf.at[folder, "dice"] = m_test_dice
    resdf.at[folder, "dice_sd"] = np.std(test_dice)


In [41]:
a = [1,7,6,2,4]
np.std(a)

2.280350850198276

In [42]:
resdf = pd.DataFrame(index=folders)
for f in folders:
    evaluate(f, resdf, dataloaders)

resdf

revacc_0_unet-6v.pt
.............................................................................................................................................
Scores on 141 test images: 
avg test loss: 0.09045835119401309
avg test jaccard: 0.8858040892486031
avg test dice: 0.9384132551808729
revacc_1_unet-6v.pt
.............................................................................................................................................
Scores on 141 test images: 
avg test loss: 0.10624498371959579
avg test jaccard: 0.8793780562725473
avg test dice: 0.9344203455228333
revacc_2_unet-6v.pt
.............................................................................................................................................
Scores on 141 test images: 
avg test loss: 0.09549594509369093
avg test jaccard: 0.8883732464296598
avg test dice: 0.9397749723272121
revacc_3_unet-6v.pt
............................................................................................

,loss,loss_sd,jaccard,jaccard_sd,dice,dice_sd
revacc_0,0.090458,0.055986,0.885804,0.056987,0.938413,0.034176
revacc_1,0.106245,0.076266,0.879378,0.065470,0.934420,0.040211
revacc_2,0.095496,0.066439,0.888373,0.059553,0.939775,0.035318
revacc_3,0.093081,0.068119,0.884558,0.055262,0.937792,0.032459


In [49]:
resdf.round(4)

,loss,loss_sd,jaccard,jaccard_sd,dice,dice_sd
revacc_0,0.0905,0.0560,0.8858,0.0570,0.9384,0.0342
revacc_1,0.1062,0.0763,0.8794,0.0655,0.9344,0.0402
revacc_2,0.0955,0.0664,0.8884,0.0596,0.9398,0.0353
revacc_3,0.0931,0.0681,0.8846,0.0553,0.9378,0.0325
